<a href="https://colab.research.google.com/github/daothuphuong98/COVID19-NER/blob/main/PHOBERT_BASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/VinAIResearch/PhoNER_COVID19.git

Cloning into 'PhoNER_COVID19'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 58 (delta 23), reused 41 (delta 18), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [ ]:
!pip install datasets pytorch-crf==0.7.2 seqeval transformers tensorflow_addons git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-dklsvnei
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-dklsvnei
     |████████████████████████████████| 452 kB 36.0 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 5.8 MB 44.6 MB/s 
     |████████████████████████████████| 1.1 MB 34.3 MB/s 
     |████████████████████████████████| 182 kB 63.5 MB/s 
     |████████████████████████████████| 132 kB 56.0 MB/s 
     |████████████████████████████████| 212 kB 62.8 MB/s 
     |████████████████████████████████| 127 kB 69.9 MB/s 
     |████████████████████████████████| 7.6 MB 66.5 MB/s 
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101076 sha256=01bfddf7ebfc3b305b69b058e3d2c65d3200fe009a388dbb69884ccd027d2f7d
  Stored in 

In [ ]:
import pandas as pd
import numpy as np
import re
import torch
import tensorflow as tf
import torch.nn as nn
from torchcrf import CRF
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertForTokenClassification, AdamW, XLMRobertaTokenizer, XLMRobertaConfig, XLMRobertaModel, TrainingArguments, AutoTokenizer, AutoConfig, Trainer, AutoModel, DataCollatorForTokenClassification
from tqdm import tqdm, trange, tqdm_notebook
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel
from seqeval.metrics import f1_score, classification_report
from datasets import ClassLabel, Dataset, DatasetDict
# from transformers import get_linear_schedule_with_warmup, get_constant_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup
pd.set_option('display.max_colwidth', -1)

<ipython-input-4-f82015fadac9>:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device)

MAX_LEN = 256
BS = 32

linear_dropout = 0.3
bert_att_dropout = 0.15
bert_hidd_dropout = 0.2

bert_lr = 1e-5
bert_weight_decay = 1e-5
softmax_lr = 1e-3
softmax_weight_decay = 1e-3

cuda


In [ ]:
!nvidia-smi

Sat Dec 24 08:41:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    28W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Data

## Words

In [ ]:
import pandas as pd
import re
import plotly.express as px

In [ ]:
def handle_file(link):
    with open(link) as f:
        contents = f.readlines()
    idx_sentence, idx_arr, word, tag = 0, [], [], []
    for line in contents:
        arr_info = line.split(' ')
        # print(arr_info)
        if len(arr_info) == 1:
            idx_sentence += 1
            continue
        word.append(arr_info[0])
        tag.append(arr_info[1].replace('\n', ''))
        idx_arr.append(idx_sentence)
    return pd.DataFrame(
        data = {
            'idx': idx_arr,
            'word': word,
            'tag': tag
        }
    )

In [ ]:
word_train_data = handle_file('/content/PhoNER_COVID19/data/word/train_word.conll')
word_test_data = handle_file('/content/PhoNER_COVID19/data/word/test_word.conll')
word_val_data = handle_file('/content/PhoNER_COVID19/data/word/dev_word.conll')

word_train_data['word_process'] = word_train_data['word'].apply(lambda x: re.sub('[0-9]+', "<NUM>", x).lower())
word_test_data['word_process'] = word_test_data['word'].apply(lambda x: re.sub('[0-9]+', "<NUM>", x).lower())
word_val_data['word_process'] = word_val_data['word'].apply(lambda x: re.sub('[0-9]+', "<NUM>", x).lower())

filter_string = '?!"/\{}().,'
word_train_data = word_train_data[word_train_data['word_process'].apply(lambda x: x not in filter_string)]
word_test_data = word_test_data[word_test_data['word_process'].apply(lambda x: x not in filter_string)]
word_val_data = word_val_data[word_val_data['word_process'].apply(lambda x: x not in filter_string)]

word_train_data_group = word_train_data.groupby('idx').agg({'word_process': ' '.join, 'word': ' '.join, 'tag': list}) 
word_test_data_group = word_test_data.groupby('idx').agg({'word_process': ' '.join, 'word': ' '.join, 'tag': list}) 
word_val_data_group = word_val_data.groupby('idx').agg({'word_process': ' '.join, 'word': ' '.join, 'tag': list}) 

word_train_data_group['len_tag'] = word_train_data_group['tag'].apply(lambda x: len(x))
word_val_data_group['len_tag'] = word_val_data_group['tag'].apply(lambda x: len(x))
word_test_data_group['len_tag'] = word_test_data_group['tag'].apply(lambda x: len(x))

word_train_data_group['tag_string'] = word_train_data_group['tag'].apply(lambda x: ' '.join(x))
word_val_data_group['tag_string'] = word_val_data_group['tag'].apply(lambda x: ' '.join(x))
word_test_data_group['tag_string'] = word_test_data_group['tag'].apply(lambda x: ' '.join(x))

# train_data_group = train_data_group[train_data_group['len_tag'] <= 55].reset_index()
# val_data_group = val_data_group[val_data_group['len_tag'] <= 55].reset_index()
# test_data_group = test_data_group[test_data_group['len_tag'] <= 55].reset_index()

In [ ]:
word_train_data_group[word_train_data_group['len_tag'] < 15].sample(20)

,word_process,word,tag,len_tag,tag_string
idx,,,,,
4901,ngày <num>/<num> bệnh_nhân đến bệnh_viện đa_khoa tỉnh ninh_thuận khám và điều_trị,Ngày 15/3 bệnh_nhân đến Bệnh_viện đa_khoa tỉnh Ninh_Thuận khám và điều_trị,"[O, B-DATE, O, O, B-LOCATION, I-LOCATION, I-LOCATION, I-LOCATION, O, O, O]",11,O B-DATE O O B-LOCATION I-LOCATION I-LOCATION I-LOCATION O O O
4295,những bệnh_nhân suy thận như oanh không không_thể rời xa bệnh_viện quá hai ngày,Những bệnh_nhân suy thận như Oanh không không_thể rời xa bệnh_viện quá hai ngày,"[O, O, B-SYMPTOM_AND_DISEASE, I-SYMPTOM_AND_DISEASE, O, B-NAME, O, O, O, O, O, O, O, O]",14,O O B-SYMPTOM_AND_DISEASE I-SYMPTOM_AND_DISEASE O B-NAME O O O O O O O O
511,ngày <num>/<num> bệnh_nhân sốt cao không hạ,Ngày 16/8 bệnh_nhân sốt cao không hạ,"[O, B-DATE, O, B-SYMPTOM_AND_DISEASE, I-SYMPTOM_AND_DISEASE, O, O]",7,O B-DATE O B-SYMPTOM_AND_DISEASE I-SYMPTOM_AND_DISEASE O O
1237,kết_quả xét_nghiệm sàng_lọc của trung_tâm kiểm_soát bệnh_tật hà_nội âm_tính cách_ly tập_trung ở tỉnh bắc_giang,Kết_quả xét_nghiệm sàng_lọc của Trung_tâm Kiểm_soát Bệnh_tật Hà_Nội âm_tính cách_ly tập_trung ở tỉnh Bắc_Giang,"[O, O, O, O, B-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, O, O, O, O, B-LOCATION, I-LOCATION]",14,O O O O B-ORGANIZATION I-ORGANIZATION I-ORGANIZATION I-ORGANIZATION O O O O B-LOCATION I-LOCATION
2859,hiện_tình_trạng bệnh_nhân khá nặng mệt nhiều khó thở đau ngực,Hiện_tình_trạng bệnh_nhân khá nặng mệt nhiều khó thở đau ngực,"[O, O, O, O, B-SYMPTOM_AND_DISEASE, I-SYMPTOM_AND_DISEASE, B-SYMPTOM_AND_DISEASE, I-SYMPTOM_AND_DISEASE, B-SYMPTOM_AND_DISEASE, I-SYMPTOM_AND_DISEASE]",10,O O O O B-SYMPTOM_AND_DISEASE I-SYMPTOM_AND_DISEASE B-SYMPTOM_AND_DISEASE I-SYMPTOM_AND_DISEASE B-SYMPTOM_AND_DISEASE I-SYMPTOM_AND_DISEASE
1923,ngày <num>/<num> ông biểu_hiện sốt nhẹ nhập_viện tại bệnh_viện đa_khoa tỉnh vĩnh_phúc,Ngày 28/3 ông biểu_hiện sốt nhẹ nhập_viện tại Bệnh_viện Đa_khoa tỉnh Vĩnh_Phúc,"[O, B-DATE, O, O, B-SYMPTOM_AND_DISEASE, I-SYMPTOM_AND_DISEASE, O, O, B-LOCATION, I-LOCATION, I-LOCATION, I-LOCATION]",12,O B-DATE O O B-SYMPTOM_AND_DISEASE I-SYMPTOM_AND_DISEASE O O B-LOCATION I-LOCATION I-LOCATION I-LOCATION
1333,bác_sĩ huỳnh_thái_tiễn thăm_khám cho bệnh_nhân tại bệnh_viện điều_trị covid - <num> ở cần_giờ,Bác_sĩ Huỳnh_Thái_Tiễn thăm_khám cho bệnh_nhân tại bệnh_viện điều_trị COVID - 19 ở Cần_Giờ,"[O, O, O, O, O, O, B-LOCATION, I-LOCATION, I-LOCATION, I-LOCATION, I-LOCATION, I-LOCATION, I-LOCATION]",13,O O O O O O B-LOCATION I-LOCATION I-LOCATION I-LOCATION I-LOCATION I-LOCATION I-LOCATION
845,cô từ mỹ_về quá_cảnh ở philippines nhập_cảnh việt_nam ngày <num> - <num>,Cô từ Mỹ_về quá_cảnh ở Philippines nhập_cảnh Việt_Nam ngày 13 - 3,"[O, O, B-LOCATION, O, O, B-LOCATION, O, B-LOCATION, O, B-DATE, I-DATE, I-DATE]",12,O O B-LOCATION O O B-LOCATION O B-LOCATION O B-DATE I-DATE I-DATE
4530,trong thời_gian ông chuyển sang điều_trị tại khoa a<num> người thân_quen liên_tục đến thăm,Trong thời_gian ông chuyển sang điều_trị tại khoa A2 người thân_quen liên_tục đến thăm,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]",14,O O O O O O O O O O O O O O


#MODEL

In [ ]:
pre_train = {
    'mbert': 'bert-base-multilingual-cased',
    'xlmr_base': 'xlm-roberta-base',
    'xlmr_large': 'xlm-roberta-large',
    'phobert': 'vinai/phobert-base'
}
pre_trained_name = pre_train['phobert']
MAX_LEN = 128

In [ ]:
tag_values = ['O',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'B-SYMPTOM_AND_DISEASE',
 'I-SYMPTOM_AND_DISEASE',
 'B-LOCATION',
 'B-DATE',
 'B-PATIENT_ID',
 'B-AGE',
 'B-NAME',
 'I-DATE',
 'B-JOB',
 'I-LOCATION',
 'B-TRANSPORTATION',
 'B-GENDER',
 'I-TRANSPORTATION',
 'I-JOB',
 'I-NAME',
 'I-AGE',
 'I-PATIENT_ID']
tags = ClassLabel(num_classes=len(tag_values), names=tag_values)

In [ ]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}
tokenizer = AutoTokenizer.from_pretrained(pre_trained_name, do_lower_case=False,use_fast=True)
config = AutoConfig.from_pretrained(pre_trained_name, output_hidden_states=False,
                                    id2label=index2tag, label2id=tag2index)
# model = AutoModel.from_pretrained(pre_trained_name, config=config)
config.hidden_dropout_prob = bert_hidd_dropout
config.attention_probs_dropout_prob = bert_att_dropout

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Without CRF

In [ ]:
def align_predictions(predictions, label_ids):
  preds = np.argmax(predictions, axis=2)
  batch_size, seq_len = preds.shape
  labels_list, preds_list = [], []
  for batch_idx in range(batch_size):
    example_labels, example_preds = [], []
    for seq_idx in range(seq_len):
      if label_ids[batch_idx, seq_idx] != -100:
        example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
        example_preds.append(index2tag[preds[batch_idx][seq_idx]])
    labels_list.append(example_labels)
    preds_list.append(example_preds)
  return preds_list, labels_list

In [ ]:
def get_word_ids(encoded, tokenized):
  word_ids = [None] * len(encoded)
  prev_flag = 0
  cnt = 0
  for ind, (x,y) in enumerate(zip(encoded[1:], tokenized)):
    if x > 2:
      word_ids[ind+1] = cnt
      if y[-2:] == '@@':
        prev_flag = 1
      else:
        prev_flag = 0
      if prev_flag == 0:
        cnt += 1
  return word_ids

def encode(r):
  X = tokenizer(r['word_process'], 
                truncation=True,
                padding='max_length', 
                max_length=MAX_LEN)
  tokenized = tokenizer.tokenize(r['word_process']) 
  previous_word_idx = None
  label_ids = []
  word_ids = get_word_ids(X['input_ids'], tokenized)
  for word_idx in word_ids:
    if word_idx is None:
      label_ids.append(0)
    else:
      label_ids.append(tag2index[r['tag'][word_idx]])
    previous_word_idx = word_idx
  return X.input_ids, label_ids

def preprocess(df):
  df[['tokenized_sentence','tokenized_labels']] = df.apply(lambda x: encode(x),  axis=1, result_type='expand')
  return df

word_train_data_group = preprocess(word_train_data_group)
word_val_data_group = preprocess(word_val_data_group)
word_test_data_group = preprocess(word_test_data_group)

In [ ]:
datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(word_train_data_group[['tokenized_sentence', 'tokenized_labels']].rename(columns={'tokenized_sentence': 'input_ids',
                                                                                  'tokenized_labels': 'labels'})),
    "valid": Dataset.from_pandas(word_val_data_group[['tokenized_sentence', 'tokenized_labels']].rename(columns={'tokenized_sentence': 'input_ids',
                                                                                  'tokenized_labels': 'labels'})),
    "test": Dataset.from_pandas(word_test_data_group[['tokenized_sentence', 'tokenized_labels']].rename(columns={'tokenized_sentence': 'input_ids',
                                                                                  'tokenized_labels': 'labels'}))
    })
datasets_train_test

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'idx'],
        num_rows: 5027
    })
    valid: Dataset({
        features: ['input_ids', 'labels', 'idx'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'idx'],
        num_rows: 3000
    })
})

In [ ]:
class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
  config_class = XLMRobertaConfig
  def __init__(self, config, num_layer=197):
    super().__init__(config)
    self.num_labels = config.num_labels
    # Load model body
    self.roberta = RobertaModel(config, add_pooling_layer=False)
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)
    # Load and initialize weights
    self.init_weights()
  def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,labels=None, **kwargs):
    # Use model body to get encoder representations
    outputs = self.roberta(input_ids, attention_mask=attention_mask,
    token_type_ids=token_type_ids, **kwargs)
    # Apply classifier to encoder representation
    sequence_output = self.dropout(outputs[0])
    logits = self.classifier(sequence_output)
    # Calculate losses
    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    # Return model output object
    return TokenClassifierOutput(loss=loss, logits=logits,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions)

## With CRF

In [ ]:
def align_predictions(predictions, label_ids):
  preds = predictions
  batch_size, seq_len = preds.shape
  labels_list, preds_list = [], []
  for batch_idx in range(batch_size):
    example_labels, example_preds = [], []
    for seq_idx in range(seq_len):
      if label_ids[batch_idx, seq_idx] != -100:
        example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
        example_preds.append(index2tag[preds[batch_idx][seq_idx]])
    labels_list.append(example_labels)
    preds_list.append(example_preds)
  return preds_list, labels_list

def compute_metrics(eval_pred):
  print(eval_pred)
  y_pred, y_true = align_predictions(eval_pred.predictions,
                                     eval_pred.label_ids)
  return {"f1": f1_score(y_true, y_pred)}

In [ ]:
def get_word_ids(encoded, tokenized):
  word_ids = [None] * len(encoded)
  prev_flag = 0
  cnt = 0
  for ind, (x,y) in enumerate(zip(encoded[1:], tokenized)):
    if x > 2:
      word_ids[ind+1] = cnt
      if y[-2:] == '@@':
        prev_flag = 1
      else:
        prev_flag = 0
      if prev_flag == 0:
        cnt += 1
  return word_ids

def encode(r):
  X = tokenizer(r['word_process'], 
                truncation=True,
                padding='max_length', 
                max_length=MAX_LEN)
  tokenized = tokenizer.tokenize(r['word_process']) 
  previous_word_idx = None
  label_ids = []
  word_ids = get_word_ids(X['input_ids'], tokenized)  
  label_ids_mask = []
  for word_idx in word_ids:
    if word_idx is None :
      label_ids.append(0)
    else:
      label_ids.append(tag2index[r['tag'][word_idx]])
    
    if word_idx is None or word_idx == previous_word_idx:
      label_ids_mask.append(False)
    elif word_idx != previous_word_idx:
      label_ids_mask.append(True)
    previous_word_idx = word_idx
  return X.input_ids, label_ids, label_ids_mask

def preprocess(df):
  df[['tokenized_sentence','tokenized_labels', 'label_ids_mask']] = df.apply(lambda x: encode(x),  axis=1, result_type='expand')
  return df

word_train_data_group = preprocess(word_train_data_group)
word_val_data_group = preprocess(word_val_data_group)
word_test_data_group = preprocess(word_test_data_group)

In [ ]:
datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(word_train_data_group[['tokenized_sentence', 'tokenized_labels']].rename(columns={'tokenized_sentence': 'input_ids',
                                                                                  'tokenized_labels': 'labels'})),
    "valid": Dataset.from_pandas(word_val_data_group[['tokenized_sentence', 'tokenized_labels']].rename(columns={'tokenized_sentence': 'input_ids',
                                                                                  'tokenized_labels': 'labels'})),
    "test": Dataset.from_pandas(word_test_data_group[['tokenized_sentence', 'tokenized_labels']].rename(columns={'tokenized_sentence': 'input_ids',
                                                                                  'tokenized_labels': 'labels'}))
    })
datasets_train_test

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'idx'],
        num_rows: 5027
    })
    valid: Dataset({
        features: ['input_ids', 'labels', 'idx'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'idx'],
        num_rows: 3000
    })
})

In [ ]:
class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
  config_class = XLMRobertaConfig
  def __init__(self, config, num_layer=197):
    super().__init__(config)
    self.num_labels = config.num_labels
    # Load model body
    self.roberta = RobertaModel(config, add_pooling_layer=False)
    # Set up token classification head
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)
    self.crf = CRF(num_tags=self.num_labels)
    # Load and initialize weights
    self.init_weights()
  def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,labels=None, **kwargs):
    outputs = self.roberta(input_ids=input_ids,
                        token_type_ids=token_type_ids,
                        attention_mask=attention_mask)
    sequence_output = self.dropout(outputs[0])
    emissions = self.classifier(sequence_output)

    if labels is not None:
        log_likelihood = self.crf(emissions, labels)
        sequence_of_tags = torch.Tensor(self.crf.decode(emissions))
        return TokenClassifierOutput(loss=-log_likelihood, logits=torch.Tensor(sequence_of_tags).T)
    else:
        sequence_of_tags = self.crf.decode(emissions)
        return sequence_of_tags

## Run model

In [ ]:
num_epochs = 10
batch_size = 8

def model_init():
  return (XLMRobertaForTokenClassification.from_pretrained(pre_trained_name, config=config).to(device))

training_args = TrainingArguments(
      output_dir=pre_trained_name+'_output', log_level="error", num_train_epochs=num_epochs,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size, evaluation_strategy="epoch",
      logging_steps=len(datasets_train_test["train"]) // batch_size,
       save_steps=1e6, weight_decay=0.01, disable_tqdm=False, push_to_hub=False)

data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(model_init=model_init, args=training_args,
        data_collator=data_collator, compute_metrics=compute_metrics,
        train_dataset=datasets_train_test["train"],
        eval_dataset=datasets_train_test["valid"],
        tokenizer=tokenizer)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
#Without CRF
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.116000,0.042807,0.927180
2,0.024900,0.037363,0.926817
3,0.015500,0.032554,0.948148
4,0.011700,0.033013,0.952860
5,0.008900,0.036201,0.955189
6,0.006900,0.037430,0.953070
7,0.005600,0.035861,0.955184
8,0.004600,0.038534,0.956904
9,0.003600,0.038673,0.957643
10,0.003100,0.039340,0.957214


TrainOutput(global_step=6290, training_loss=0.02005262237829889, metrics={'train_runtime': 1591.3081, 'train_samples_per_second': 31.59, 'train_steps_per_second': 3.953, 'total_flos': 3284381393356800.0, 'train_loss': 0.02005262237829889, 'epoch': 10.0})

In [ ]:
#With CRF
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:413.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Epoch,Training Loss,Validation Loss,F1
1,129.115400,50.046757,0.919564
2,26.210700,37.470631,0.944728
3,18.666800,42.304386,0.953071
4,15.258100,48.948647,0.953515
5,13.750600,55.402122,0.956949
6,11.625800,60.970642,0.958472
7,10.107400,62.688400,0.958478
8,8.638800,66.585999,0.959117
9,7.118200,68.496750,0.959107
10,5.749000,71.134140,0.959806


TrainOutput(global_step=6290, training_loss=24.599759973668522, metrics={'train_runtime': 1726.9326, 'train_samples_per_second': 29.109, 'train_steps_per_second': 3.642, 'total_flos': 3284398380595200.0, 'train_loss': 24.599759973668522, 'epoch': 10.0})

In [ ]:
trainer.save_model('phobert-base')

# Evaluation

## Without CRF

In [ ]:
model = XLMRobertaForTokenClassification.from_pretrained('/content/drive/MyDrive/phobert-base', config=config)
trainer = Trainer(model=model)
x=trainer.predict(datasets_train_test['test'])

The following columns in the test set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: idx. If idx are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3000
  Batch size = 8


In [ ]:
pred = np.argmax(x.predictions, axis=2)
lab = np.array(datasets_train_test['test']['labels'])
preds_list, labels_list = align_predictions(x.predictions, lab)
print(classification_report(labels_list, preds_list, digits=4))

                     precision    recall  f1-score   support

                AGE     0.9546    0.9700    0.9623      1735
               DATE     0.9832    0.9954    0.9893      7656
             GENDER     0.9579    0.9795    0.9686       488
                JOB     0.7784    0.8136    0.7956       177
           LOCATION     0.9318    0.9347    0.9332      7172
               NAME     0.9372    0.9420    0.9396       983
       ORGANIZATION     0.8418    0.8737    0.8574       950
         PATIENT_ID     0.9668    0.9842    0.9754      6027
SYMPTOM_AND_DISEASE     0.8427    0.8225    0.8325      1172
     TRANSPORTATION     0.9664    0.9942    0.9801       694

          micro avg     0.9492    0.9600    0.9546     27054
          macro avg     0.9161    0.9310    0.9234     27054
       weighted avg     0.9491    0.9600    0.9545     27054



## With CRF

In [ ]:
model = XLMRobertaForTokenClassification.from_pretrained('/content/drive/MyDrive/phobert-base-crf', config=config)
trainer = Trainer(model=model)
x=trainer.predict(datasets_train_test['test'])

loading weights file /content/drive/MyDrive/phobert-base-crf/pytorch_model.bin
All model checkpoint weights were used when initializing XLMRobertaForTokenClassification.

All the weights of XLMRobertaForTokenClassification were initialized from the model checkpoint at /content/drive/MyDrive/phobert-base-crf.
If your task is similar to the task the model of the checkpoint was trained on, you can already use XLMRobertaForTokenClassification for predictions without further training.
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: idx. If idx a

In [ ]:
mask=np.array(word_test_data_group['label_ids_mask'].values.tolist())
lab = np.array(datasets_train_test['test']['labels'])
labs=[]
for ind, l in enumerate(lab):
  labs.append(np.where(mask[ind], l, -100))
labs = np.array(labs).astype('int')

In [ ]:
preds_list, labels_list = align_predictions(x.predictions, labs)
print(classification_report(labels_list, preds_list, digits=4))

                     precision    recall  f1-score   support

                AGE     0.9572    0.9671    0.9621       578
               DATE     0.9742    0.9848    0.9795      1647
             GENDER     0.9697    0.9782    0.9739       458
                JOB     0.8218    0.8266    0.8242       173
           LOCATION     0.9319    0.9325    0.9322      4430
               NAME     0.9538    0.9146    0.9338       316
       ORGANIZATION     0.8617    0.8807    0.8711       771
         PATIENT_ID     0.9744    0.9739    0.9741      1990
SYMPTOM_AND_DISEASE     0.8441    0.8389    0.8415      1136
     TRANSPORTATION     0.9458    0.9948    0.9697       193

          micro avg     0.9338    0.9369    0.9354     11692
          macro avg     0.9235    0.9292    0.9262     11692
       weighted avg     0.9338    0.9369    0.9353     11692

